## Covering all model components
* mean model --> so far: Zero mean, try out new models: Constant mean and ARX
* volatility model --> GARCH(1,1)
* distribution --> normal distributed

In [47]:
from arch import arch_model
from datetime import timedelta
import pandas as pd
import numpy as np

from evaluation.help_functions.prepare_data import most_recent_thursday, next_working_days
from dax.help_functions.calculate_returns import calculate_returns
from dax.help_functions.get_quantiles import get_norm_quantiles
from dax.help_functions.get_dax_data import get_data


daxdata = get_data().iloc[8000:,]
start_date_excl = most_recent_thursday(daxdata) - timedelta(days=1)
daxdata = daxdata.loc[daxdata.index <= start_date_excl]
date_st = start_date_excl.strftime('%Y-%m-%d')

daxdata = calculate_returns(daxdata, 5)
daxdata.index = daxdata.index.date
daxdata = daxdata.dropna()
daxdata = daxdata[['Close', 'LogRetLag1',
                   'LogRetLag2', 'LogRetLag3', 'LogRetLag4', 'LogRetLag5']]
daxdata

,Close,LogRetLag1,LogRetLag2,LogRetLag3,LogRetLag4,LogRetLag5
2019-09-10,12268.709961,0.347914,0.629422,1.163586,2.006097,2.960150
2019-09-11,12359.070312,0.733812,1.081725,1.363234,1.897398,2.739909
2019-09-12,12410.250000,0.413251,1.147063,1.494977,1.776485,2.310649
2019-09-13,12468.530273,0.468515,0.881766,1.615578,1.963491,2.245000
2019-09-16,12380.309570,-0.710062,-0.241547,0.171704,0.905516,1.253429
...,...,...,...,...,...,...
2023-11-16,15786.610352,0.243797,1.096667,2.837245,3.560679,2.788120
2023-11-17,15919.160156,0.836129,1.079926,1.932796,3.673374,4.396808
2023-11-20,15901.330078,-0.112067,0.724062,0.967860,1.820729,3.561307
2023-11-21,15900.530273,-0.005030,-0.117097,0.719032,0.962830,1.815699


In [48]:
import pmdarima
import statsmodels as sm
import arch

# fit ARIMA on returns
arima_model_fitted = pmdarima.auto_arima(daxdata['LogRetLag1'])
arima_model_fitted.order

(0, 0, 0)

Auto ARIMA model tells us that best mean is 0 mean --> try out ARMA(1,1) nevertheless

In [65]:
model = arch_model(daxdata['LogRetLag1'], mean='AR', p=1, q=1)
model_fit = model.fit()

predictions = model_fit.forecast(horizon=5)
np.array(predictions.mean)[0][-1]
#predictions.variance.values

Iteration:      1,   Func. Count:      6,   Neg. LLF: 20221.713671609155
Iteration:      2,   Func. Count:     15,   Neg. LLF: 425705458476.4596
Iteration:      3,   Func. Count:     23,   Neg. LLF: 2121.3041844436784
Iteration:      4,   Func. Count:     30,   Neg. LLF: 2123.8542260892054
Iteration:      5,   Func. Count:     36,   Neg. LLF: 1685.4298317703115
Iteration:      6,   Func. Count:     42,   Neg. LLF: 1682.7014064906982
Iteration:      7,   Func. Count:     47,   Neg. LLF: 1682.6865026658477
Iteration:      8,   Func. Count:     52,   Neg. LLF: 1682.6858515030933
Iteration:      9,   Func. Count:     57,   Neg. LLF: 1682.685794492621
Iteration:     10,   Func. Count:     62,   Neg. LLF: 1682.6857597398964
Iteration:     11,   Func. Count:     66,   Neg. LLF: 1682.6857597394705
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1682.6857597398964
            Iterations: 11
            Function evaluations: 66
            Gradient evalu

0.06225480277754374

In [56]:
# ARMA(1,1) model using statsmodels
arma_model = pmdarima.arima.ARIMA(order=(1, 0, 1))
arma_results = arma_model.fit(daxdata['LogRetLag1'])

# GARCH(1,1) model using arch
garch_model = arch_model(daxdata['LogRetLag1'],p=1, q=1)
garch_results = garch_model.fit()

arma_results.predict(5)
garch_results.predict(5)

c:\Users\Maria\Documents\Setups\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\Users\Maria\Documents\Setups\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)


Iteration:      1,   Func. Count:      6,   Neg. LLF: 20221.713671609155
Iteration:      2,   Func. Count:     15,   Neg. LLF: 425705458476.4596
Iteration:      3,   Func. Count:     23,   Neg. LLF: 2121.3041844436784
Iteration:      4,   Func. Count:     30,   Neg. LLF: 2123.8542260892054
Iteration:      5,   Func. Count:     36,   Neg. LLF: 1685.4298317703115
Iteration:      6,   Func. Count:     42,   Neg. LLF: 1682.7014064906982
Iteration:      7,   Func. Count:     47,   Neg. LLF: 1682.6865026658477
Iteration:      8,   Func. Count:     52,   Neg. LLF: 1682.6858515030933
Iteration:      9,   Func. Count:     57,   Neg. LLF: 1682.685794492621
Iteration:     10,   Func. Count:     62,   Neg. LLF: 1682.6857597398964
Iteration:     11,   Func. Count:     66,   Neg. LLF: 1682.6857597394705
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1682.6857597398964
            Iterations: 11
            Function evaluations: 66
            Gradient evalu

c:\Users\Maria\Documents\Setups\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
c:\Users\Maria\Documents\Setups\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: FutureWarning: No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.
  return get_prediction_index(


AttributeError: 'ARCHModelResult' object has no attribute 'predict'

In [40]:
arma_model = pmdarima.arima.ARIMA(order=(1,0,1))
arma_model.fit(daxdata['LogRetLag1'])


# Use ARMA to predict mu --> mean component
predicted_mu = arma_model.predict(n_periods=1)

# fit a GARCH(1,1) model on the residuals of the ARMA model
garch = arch.arch_model(arma_residuals, p=1, q=1)
garch_fitted = garch.fit()

# Use GARCH to predict the residual --> variance component 
garch_variance = garch_fitted.forecast(horizon=1)
predicted_et = garch_variance.mean['h.1'].iloc[-1]

print(garch_variance)
# Combine both models' output: yt = mu + et
prediction = predicted_mu + predicted_et
print(prediction)

c:\Users\Maria\Documents\Setups\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\Users\Maria\Documents\Setups\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)


Iteration:      1,   Func. Count:      6,   Neg. LLF: 21329.154582094467
Iteration:      2,   Func. Count:     15,   Neg. LLF: 401485083641.05475
Iteration:      3,   Func. Count:     23,   Neg. LLF: 2118.9392873926745
Iteration:      4,   Func. Count:     30,   Neg. LLF: 2151.5869944546585
Iteration:      5,   Func. Count:     36,   Neg. LLF: 1683.724922313428
Iteration:      6,   Func. Count:     42,   Neg. LLF: 1681.7348762545657
Iteration:      7,   Func. Count:     47,   Neg. LLF: 1681.7196036188511
Iteration:      8,   Func. Count:     52,   Neg. LLF: 1681.719385786248
Iteration:      9,   Func. Count:     57,   Neg. LLF: 1681.719354752845
Iteration:     10,   Func. Count:     62,   Neg. LLF: 1681.719340292042
Iteration:     11,   Func. Count:     66,   Neg. LLF: 1681.7193402909604
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1681.719340292042
            Iterations: 11
            Function evaluations: 66
            Gradient evaluati

c:\Users\Maria\Documents\Setups\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
c:\Users\Maria\Documents\Setups\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: FutureWarning: No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.
  return get_prediction_index(


In [23]:

arima_residuals = arima_model.arima_res_.resid

# fit a GARCH(1,1) model on the residuals of the ARIMA model
garch = arch.arch_model(arima_residuals, p=1, q=1)
garch_fitted = garch.fit()

# Use ARIMA to predict mu
predicted_mu = arima_model.predict(n_periods=1)[0]
# Use GARCH to predict the residual
garch_forecast = garch_model.forecast(horizon=1)
predicted_et = garch_forecast.mean['h.1'].iloc[-1]
# Combine both models' output: yt = mu + et
prediction = predicted_mu + predicted_et

TypeError: ARIMA.fit() missing 1 required positional argument: 'y'

AR(2) passt noch am besten, aber alle Modelle schlecht --> trotzdem mal ausprobieren

In [19]:
from arch.univariate import ARX, GARCH

# mean component 
mean_model_ar2 = ARX(daxdata['LogRetLag1'], lags=[2])

# volatility component 
mean_model_ar2.volatility = GARCH(p=1, q=1)

result = mean_model_ar2.fit(update_freq=0, disp="off")

forecast = result.forecast(horizon=1, reindex=False)
forecast.variance

,h.1
2023-11-22,0.717912
